In [1]:
import torchvision as thv
import numpy as np
import os

In [2]:
# Doanload Data
data_exists = os.path.isdir("./MNIST")
train = thv.datasets.MNIST('./', download=data_exists, train=True)
val = thv.datasets.MNIST('./', download=data_exists, train=False)

print(train.data.shape, len(train.targets))

torch.Size([60000, 28, 28]) 60000


In [3]:
class layer_t:
    def backward_check(self, dh_l1):
        # PS: since the matrix dw is really sparse, it's good to try and visualize it
        # for picking i,j values or try many different values
        i, j = 7, 432

        k = np.argwhere(dh_l1 == 1)[0, 1]
        epsilon_ij = np.random.normal(0, 1)
        epsilon = np.zeros_like(self.w)
        epsilon[i, j] = epsilon_ij

        dw_ij_num = np.matmul((self.w + epsilon), self.hl.T)[k, 0] - np.matmul((self.w - epsilon), self.hl.T)[k, 0]
        dw_ij_den = 2 * epsilon_ij

        dw_ij = dw_ij_num / dw_ij_den

        return dw_ij, self.dw[i,j]

    
#     def backward_check_h(self, dh_l1, i=7):
#         # PS: since the matrix hl is really sparse, it's good to try and visualize it
#         # for picking i values or try many different values
#         k = np.argwhere(dh_l1 == 1)[0, 1]
        
#         epsilon_i = np.random.normal(0, 1)
#         epsilon = np.zeros_like(self.hl)
#         epsilon[0, i] = epsilon_i

#         dw_i_num = np.matmul((self.hl + epsilon), self.hl.T) - np.matmul((self.hl - epsilon), self.hl.T)
#         dw_i_den = 2 * epsilon_i

#         dw_i = dw_i_num / dw_i_den
        
#         return dw_i, self.hl[0, k]
    
#     def backward_check_b(self, dh_l1, i=7):
#         # PS: since the matrix hl is really sparse, it's good to try and visualize it
#         # for picking i values or try many different valuesgo
#         k = np.argwhere(dh_l1 == 1)[0, 1]
        
#         epsilon_i = np.random.normal(0, 1)
#         epsilon = np.zeros_like(self.b)
#         epsilon[i] = epsilon_i

#         dw_i_num = np.matmul((self.b + epsilon), self.hl)[k,0] - np.matmul((self.b - epsilon), self.hl)[k,0]
#         dw_i_den = 2 * epsilon_i

#         dw_i = dw_i_num / dw_i_den
        
#         return dw_i, self.hl[0, k]

    def zero_grad(self):
        # useful to delete the stored backprop gradients of the
        # previous mini-batch before you start a new mini-batch
        # --- only used in linear layer
        pass

In [4]:
class linear_t(layer_t):
    def __init__(self):
        # input size
        self.A = 784 
        
        # class count
        self.C = 10
        
        # Fix seed
        np.random.seed(546)
        
        # Define normalized w, and b
        self.w = np.random.normal(loc=0, scale=1, size=(self.C, self.A))
        self.w = self.w / np.linalg.norm(self.w)

        self.b = np.random.normal(loc=0, scale=1, size=(self.C, 1))
        self.b = self.b / np.linalg.norm(self.b)
        
        # Define placeholder gradients to be filled in backward step
        self.dw = np.zeros_like(self.w)
        self.db = np.zeros_like(self.b)
        

    def forward(self, h_l):
        h_l1 = np.matmul(h_l, np.transpose(self.w)) + self.b.T

        # cache hˆl in forward because we will need it to compute
        # dw in backward
        self.hl = h_l
        
        return h_l1


    def backward(self, dh_l1):
        dh_l = np.matmul(dh_l1, self.w)
        dw = np.matmul(dh_l1.T, self.hl)
        db = np.matmul(dh_l1.T, np.ones([self.hl.shape[0], 1]))

        self.dw, self.db = dw, db
    
        # notice that there is no need to cache dh_{l+1}
        return dh_l
    
    def zero_grad(self):
        # useful to delete the stored backprop gradients of the
        # previous mini-batch before you start a new mini-batch
        self.dw, self.db = 0 * self.dw, 0 * self.db

In [5]:
class relu_t(layer_t):
    def forward(self, h_l):
        return np.maximum(0, h_l)
    
    def backward(self, dh_l1):
        x = np.array(dh_l1, copy=True)
        x[x <= 0] = 0
        x[x > 0] = 1
        return x

In [6]:
class softmax_cross_entropy_t(layer_t):
    def __init__(self):
        # no parameters, nothing to initialize
        self.h_l1 = None
        self.y = None

    def forward(self, h_l, y):
        expd = np.exp(h_l)
        h_l1 = expd / np.sum(expd, axis=1,keepdims=True) 
        
        self.h_l1 = h_l1
        self.y = y
       
        # compute average loss ell(y) over a mini-batch
        BATCH_SIZE = h_l1.shape[0]
        ell = np.sum(-np.log(h_l1[range(BATCH_SIZE), self.y])) / BATCH_SIZE
        
        # compute the error of predictions
        error = np.sum(np.not_equal(self.y, np.argmax(h_l1, axis=1)))/BATCH_SIZE
        
        return ell, error

    def backward(self):
        # as we saw in the notes, the backprop input to the
        # loss layer is 1, so this function does not take any
        # arguments
        
        B = self.y.shape[0] # batch size
        self.h_l1[range(B), self.y] -=1
        
        return self.h_l1/B

In [7]:
def subsample(dataset, num_samples, num_classes):
    # Get subsample by sampling
    num_samples_per_class = int(num_samples/num_classes)
    dataset = np.array(list(map(lambda elt: (np.array(elt[0]).flatten(),elt[1]), dataset)))
    X, y = map(lambda x: np.array(list(x)),zip(*dataset))
    subsamplesX, subsamplesY = None, None

    for c in range(num_classes):
        # Get all X's where label == c
        mask = np.argwhere(y == c)
        class_X, class_Y = X[mask], y[mask]
        class_X = class_X.reshape((class_X.shape[0], X.shape[1]))
        
        # Get all X's where label == c, inds = index array
        inds = np.random.randint(0, high=class_Y.shape[0], size=num_samples_per_class)
        subsamplesX = class_X[inds] if subsamplesX is None else np.concatenate([subsamplesX, class_X[inds]])
        
        # Convert Y to one-hot
#         class_subsample = np.zeros((num_samples_per_class, num_classes))
#         class_subsample[np.arange(num_samples_per_class), class_Y[inds]] = 1
#         subsamplesY = class_subsample if subsamplesY is None else np.concatenate([subsamplesY,class_subsample])

        # Get all Y's where label == c
        subsamplesY = class_Y[inds] if subsamplesY is None else np.concatenate([subsamplesY, class_Y[inds]])
    
    # shuffle the arrays
    A = subsamplesX.shape[1]
    B = subsamplesX.shape[0]
    shuf_inds = np.random.shuffle(np.array(range(B)))
    return subsamplesX[shuf_inds].reshape((B,A)), subsamplesY[shuf_inds].reshape((num_samples,))

In [8]:
# def one_hot(y_i):
#     hot_y = np.zeros((10,)) # C = 10
#     hot_y[y_i] = 1
#     return hot_y

In [9]:
######## CODE FOR CHECKING GRADIENT MATCHING #############

## THIS FAILS FOR SOME REASON ##

# l1 = linear_t()
# l2 = relu_t()
# l3 = softmax_cross_entropy_t()

# h1 = l1.forward(np.array([trainX[25064]]))
# h2 = l2.forward(h1)
# ell, error = l3.forward(h2, np.array([trainY[25064]]))


# dh2 = l3.backward()
# dh1 = l2.backward(dh2)
# dx = l1.backward(dh1)

# estimate_grad, true_grad = l1.backward_check(dh1)

# print('Estimated gradient = {}, true gradient = {}'.format(estimate_grad, true_grad))
            
#         estimate_grad, true_grad = NN.backward_check_h(dhl1, i=j)
#         if abs(estimate_grad-true_grad) > 0.003:
#             print('Estimated h_l1 = {}, true h_l1 = {}'.format(estimate_grad, true_grad))
            
#         estimate_grad, true_grad = NN.backward_check_b(dhl1, i=int(j*10/784))
#         if abs(estimate_grad-true_grad) > 0.003:
#             print('Estimated b = {}, true b = {}'.format(estimate_grad, true_grad))

In [10]:
trainX, trainY = subsample(dataset=train, num_samples=30000, num_classes=10)
# valX, valY = subsample(dataset=val, num_samples=5000, num_classes=10)
print(trainX.shape, trainY.shape)

(30000, 784) (30000,)


In [14]:
##################################################################
##################################################################
############### TRAINING THE NEURAL NETWORK ######################
##################################################################
##################################################################

l1, l2, l3 = linear_t(), relu_t(), softmax_cross_entropy_t()
net = [l1, l2, l3]

# number of iterations
iters = 50000
# learning rate
lr = 0.1

# mini-batch size
batch_size = 1

# all training data size
training_data_size = trainX.shape[0]

loss = []
# train for at least "iters" iterations
for t in range(iters):
    # 1. sample a mini-batch of size bb = 32
    # each image in the mini-batch is chosen uniformly randomly from the # training dataset
    inds = np.random.randint(0,high=training_data_size, size=batch_size)
    x, y = trainX[inds], trainY[inds]
    

    # 2. zero gradient buffer
    for l in net: 
        l.zero_grad()
        
    # 3. forward pass
    h1 = l1.forward(x)
    h2 = l2.forward(h1)
    ell, error = l3.forward(h2, y)
    
    print(f"Given true class: {y} our network outputted probabilities: {l3.h_l1}, which is class: {np.argmax(l3.h_l1)}")
    
    # 4. Backward pass
    dh2 = l3.backward()
    dh1 = l2.backward(dh2)
    dx  = l1.backward(dh1)
    
    print(f"Our network fixed it by subtracting 1 from the true class and returning: {dh2}")
    
    print(f"After ReLu layer we get:: {dh1}")
    
    
    # 5. gather backprop gradients
    dw, db = l1.dw, l1.db
    
    # 6. print some quantities for logging and debugging
    print(f"Iteration #{t} has average loss:{ell} and error: {error}")
#     print(t, np.linalg.norm(dw/l1.w), np.linalg.norm(db/l1.b))
    
    # 7. one step of SGD
    l1.w = l1.w - lr*dw
    l1.b = l1.b - lr*db
    
    loss.append(error)
    
    
#     for debugging:
    break




Given true class: [7] our network outputted probabilities: [[9.99999092e-01 5.30000495e-10 4.92321024e-10 8.86641543e-07
  4.92321024e-10 1.32360869e-08 4.92321024e-10 5.24804151e-09
  4.92321024e-10 4.92321024e-10]], which is class: 0
Our network fixed it by subtracting 1 from the true class and returning: [[ 9.99999092e-01  5.30000495e-10  4.92321024e-10  8.86641543e-07
   4.92321024e-10  1.32360869e-08  4.92321024e-10 -9.99999995e-01
   4.92321024e-10  4.92321024e-10]]
After ReLu layer we get:: [[1. 1. 1. 1. 1. 1. 1. 0. 1. 1.]]
Iteration #0 has average loss:19.06541087532226 and error: 1.0


In [12]:
plt.show(loss)

NameError: name 'plt' is not defined